In [6]:
import requests
from datetime import datetime, timedelta

# Folder where files will be saved
output_dir = "/Users/paolajaimes/Downloads"

# Start and end dates
start = datetime(2024, 1, 1)
end = datetime(2025, 4, 1)

# Function to compute last day of the month
def last_day_of_month(date):
    next_month = date.replace(day=28) + timedelta(days=4)
    return next_month - timedelta(days=next_month.day)

# Loop through months
current = start
while current <= end:
    month_end = last_day_of_month(current)
    date_str = month_end.strftime("%Y-%m-%d")       # for URL
    filename_str = month_end.strftime("%Y_%m")      # for filename
    url = f"https://www.imf.org/external/np/fin/tad/extrep2.aspx?valuedate={date_str}"

    print(f"🔄 Downloading {filename_str} from {url}")

    try:
        response = requests.get(url, timeout=30)
        if response.status_code == 200:
            # Construct TSV link and download it
            tsv_url = f"https://www.imf.org/external/np/fin/tad/extrep2.aspx?tsvflag=Y&valuedate={date_str}"
            tsv_response = requests.get(tsv_url, timeout=30)

            if tsv_response.status_code == 200:
                tsv_path = f"{output_dir}/{filename_str}.tsv"
                with open(tsv_path, "w", encoding="utf-8") as f:
                    f.write(tsv_response.text)
                print(f"✅ Saved to {tsv_path}")
            else:
                print(f"❌ Failed to get TSV for {filename_str}")
        else:
            print(f"❌ Failed to load page for {filename_str} (status {response.status_code})")
    except Exception as e:
        print(f"❌ Error for {filename_str}: {e}")

    # Move to the first of the next month
    next_month = current.replace(day=28) + timedelta(days=4)
    current = next_month.replace(day=1)

print("✅ All downloads complete.")

🔄 Downloading 2024_01 from https://www.imf.org/external/np/fin/tad/extrep2.aspx?valuedate=2024-01-31
✅ Saved to /Users/paolajaimes/Downloads/2024_01.tsv
🔄 Downloading 2024_02 from https://www.imf.org/external/np/fin/tad/extrep2.aspx?valuedate=2024-02-29
✅ Saved to /Users/paolajaimes/Downloads/2024_02.tsv
🔄 Downloading 2024_03 from https://www.imf.org/external/np/fin/tad/extrep2.aspx?valuedate=2024-03-31
✅ Saved to /Users/paolajaimes/Downloads/2024_03.tsv
🔄 Downloading 2024_04 from https://www.imf.org/external/np/fin/tad/extrep2.aspx?valuedate=2024-04-30
✅ Saved to /Users/paolajaimes/Downloads/2024_04.tsv
🔄 Downloading 2024_05 from https://www.imf.org/external/np/fin/tad/extrep2.aspx?valuedate=2024-05-31
✅ Saved to /Users/paolajaimes/Downloads/2024_05.tsv
🔄 Downloading 2024_06 from https://www.imf.org/external/np/fin/tad/extrep2.aspx?valuedate=2024-06-30
✅ Saved to /Users/paolajaimes/Downloads/2024_06.tsv
🔄 Downloading 2024_07 from https://www.imf.org/external/np/fin/tad/extrep2.aspx?va

In [34]:
pip install html5lib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 1.6 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
conda install -c conda-forge html5lib

Retrieving notices: ...working... done
Channels:
 - conda-forge
 - defaults
Platform: osx-64
Solving environment: done

## Package Plan ##

  environment location: /Users/paolajaimes/anaconda3

  added / updated specs:
    - html5lib


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2025.4.26  |       hbd8a1cb_0         149 KB  conda-forge
    certifi-2025.4.26          |     pyhd8ed1ab_0         154 KB  conda-forge
    conda-24.1.2               |  py310h2ec42d9_0         948 KB  conda-forge
    html5lib-1.1               |     pyhd8ed1ab_2          93 KB  conda-forge
    openssl-3.5.0              |       hc426f3f_1         2.6 MB  conda-forge
    python_abi-3.10            |          2_cp310           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.9 MB

The following NEW pac

In [2]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [22]:
import pandas as pd
import os

# Folder with the .tsv files
folder = "/Users/paolajaimes/Downloads"
tsv_files = sorted([f for f in os.listdir(folder) if f.endswith(".tsv")])

# Clean column names from your file layout
columns = [
    "Country",
    "GRA Disbursements", "GRA Repurchases", "GRA Charges Paid",
    "PRGT Disbursements", "PRGT Repayments", "PRGT Interest Paid",
    "RST Disbursements", "RST Repayments", "RST Interest Paid",
    "Total Disbursements", "Total Repayments", "Total Charges/Interest Paid"
]

all_data = []

for file in tsv_files:
    path = os.path.join(folder, file)
    print(f"📂 Processing {file}")

    try:
        # Skip the top 6 rows (1-indexed: rows 1–6)
        df = pd.read_csv(path, sep="\t", header=None, skiprows=6, names=columns)

        # Drop rows with missing country names
        df = df[df["Country"].notna() & (df["Country"] != "")]

        # Add month
        df["Month"] = file.replace(".tsv", "")

        # Save cleaned CSV (optional)
        out_csv = os.path.join(folder, file.replace(".tsv", ".csv"))
        df.to_csv(out_csv, index=False)

        all_data.append(df)

    except Exception as e:
        print(f"❌ Error processing {file}: {e}")

# Combine all months
if all_data:
    final = pd.concat(all_data, ignore_index=True)
    combined_path = os.path.join(folder, "imf_all_months_cleaned.csv")
    final.to_csv(combined_path, index=False)
    print(f"✅ Combined CSV saved to: {combined_path}")
else:
    print("❌ No valid tables found.")

📂 Processing 2024_01.tsv
📂 Processing 2024_02.tsv
📂 Processing 2024_03.tsv
📂 Processing 2024_04.tsv
📂 Processing 2024_05.tsv
📂 Processing 2024_06.tsv
📂 Processing 2024_07.tsv
📂 Processing 2024_08.tsv
📂 Processing 2024_09.tsv
📂 Processing 2024_10.tsv
📂 Processing 2024_11.tsv
📂 Processing 2024_12.tsv
📂 Processing 2025_01.tsv
📂 Processing 2025_02.tsv
📂 Processing 2025_03.tsv
📂 Processing 2025_04.tsv
✅ Combined CSV saved to: /Users/paolajaimes/Downloads/imf_all_months_cleaned.csv


In [1]:
## Updated download script: 2025 actuals + Jan 2026 actual + Feb–Dec 2026 projections 

import requests
from datetime import datetime, timedelta

output_dir = "/Users/paolajaimes/Downloads"

start = datetime(2025, 1, 1)
end   = datetime(2026, 1, 1)   # through Jan 2026

def last_day_of_month(date):
    next_month = date.replace(day=28) + timedelta(days=4)
    return next_month - timedelta(days=next_month.day)

current = start
while current <= end:
    month_end = last_day_of_month(current)
    date_str = month_end.strftime("%Y-%m-%d")
    filename_str = month_end.strftime("%Y_%m")

    tsv_url = f"https://www.imf.org/external/np/fin/tad/extrep2.aspx?tsvflag=Y&valuedate={date_str}"
    print(f"🔄 Downloading ACTUAL {filename_str} from {tsv_url}")

    try:
        tsv_response = requests.get(tsv_url, timeout=30)
        if tsv_response.status_code == 200:
            tsv_path = f"{output_dir}/{filename_str}.tsv"
            with open(tsv_path, "w", encoding="utf-8") as f:
                f.write(tsv_response.text)
            print(f"✅ Saved to {tsv_path}")
        else:
            print(f"❌ Failed ACTUAL {filename_str} (status {tsv_response.status_code})")
    except Exception as e:
        print(f"❌ Error ACTUAL {filename_str}: {e}")

    next_month = current.replace(day=28) + timedelta(days=4)
    current = next_month.replace(day=1)

print("✅ ACTUAL downloads complete.")


🔄 Downloading ACTUAL 2025_01 from https://www.imf.org/external/np/fin/tad/extrep2.aspx?tsvflag=Y&valuedate=2025-01-31
✅ Saved to /Users/paolajaimes/Downloads/2025_01.tsv
🔄 Downloading ACTUAL 2025_02 from https://www.imf.org/external/np/fin/tad/extrep2.aspx?tsvflag=Y&valuedate=2025-02-28
✅ Saved to /Users/paolajaimes/Downloads/2025_02.tsv
🔄 Downloading ACTUAL 2025_03 from https://www.imf.org/external/np/fin/tad/extrep2.aspx?tsvflag=Y&valuedate=2025-03-31
✅ Saved to /Users/paolajaimes/Downloads/2025_03.tsv
🔄 Downloading ACTUAL 2025_04 from https://www.imf.org/external/np/fin/tad/extrep2.aspx?tsvflag=Y&valuedate=2025-04-30
✅ Saved to /Users/paolajaimes/Downloads/2025_04.tsv
🔄 Downloading ACTUAL 2025_05 from https://www.imf.org/external/np/fin/tad/extrep2.aspx?tsvflag=Y&valuedate=2025-05-31
✅ Saved to /Users/paolajaimes/Downloads/2025_05.tsv
🔄 Downloading ACTUAL 2025_06 from https://www.imf.org/external/np/fin/tad/extrep2.aspx?tsvflag=Y&valuedate=2025-06-30
✅ Saved to /Users/paolajaimes/Do

In [3]:
import requests

output_dir = "/Users/paolajaimes/Downloads"

date1key = "2026-01-31"
year = 2026
schedule = "exp"

for mth in range(2, 13):  # Feb(2) ... Dec(12)
    filename_str = f"2026_{mth:02d}_proj"

    tsv_url = (
        "https://www.imf.org/external/np/fin/tad/extfor2.aspx"
        f"?tsvflag=Y&date1key={date1key}&mth={mth}&year={year}&schedule={schedule}"
    )

    print(f"🔄 Downloading PROJ {filename_str} from {tsv_url}")

    try:
        tsv_response = requests.get(tsv_url, timeout=30)
        if tsv_response.status_code == 200:
            tsv_path = f"{output_dir}/{filename_str}.tsv"
            with open(tsv_path, "w", encoding="utf-8") as f:
                f.write(tsv_response.text)
            print(f"✅ Saved to {tsv_path}")
        else:
            print(f"❌ Failed PROJ {filename_str} (status {tsv_response.status_code})")
    except Exception as e:
        print(f"❌ Error PROJ {filename_str}: {e}")

print("✅ PROJECTION downloads complete.")

🔄 Downloading PROJ 2026_02_proj from https://www.imf.org/external/np/fin/tad/extfor2.aspx?tsvflag=Y&date1key=2026-01-31&mth=2&year=2026&schedule=exp
✅ Saved to /Users/paolajaimes/Downloads/2026_02_proj.tsv
🔄 Downloading PROJ 2026_03_proj from https://www.imf.org/external/np/fin/tad/extfor2.aspx?tsvflag=Y&date1key=2026-01-31&mth=3&year=2026&schedule=exp
✅ Saved to /Users/paolajaimes/Downloads/2026_03_proj.tsv
🔄 Downloading PROJ 2026_04_proj from https://www.imf.org/external/np/fin/tad/extfor2.aspx?tsvflag=Y&date1key=2026-01-31&mth=4&year=2026&schedule=exp
✅ Saved to /Users/paolajaimes/Downloads/2026_04_proj.tsv
🔄 Downloading PROJ 2026_05_proj from https://www.imf.org/external/np/fin/tad/extfor2.aspx?tsvflag=Y&date1key=2026-01-31&mth=5&year=2026&schedule=exp
✅ Saved to /Users/paolajaimes/Downloads/2026_05_proj.tsv
🔄 Downloading PROJ 2026_06_proj from https://www.imf.org/external/np/fin/tad/extfor2.aspx?tsvflag=Y&date1key=2026-01-31&mth=6&year=2026&schedule=exp
✅ Saved to /Users/paolajaim

In [7]:
import pandas as pd
from pathlib import Path
from io import StringIO

# ======================
# SETTINGS
# ======================
folder = Path("/Users/paolajaimes/Downloads")   # where TSVs are saved
outdir = folder

# ======================
# READERS
# ======================
def read_imf_actual_tsv(path: Path) -> pd.DataFrame:
    """
    ACTUALS (extrep2): 13 columns, skip first 6 rows.
    """
    df = pd.read_csv(path, sep="\t", header=None, skiprows=6, dtype=str)

    df.columns = [
        "Country",
        "GRA Disbursements",
        "GRA Repurchases",
        "GRA Charges Paid",
        "PRGT Disbursements",
        "PRGT Repayments",
        "PRGT Interest Paid",
        "RST Disbursements",
        "RST Repayments",
        "RST Interest Paid",
        "Total Disbursements",
        "Total Repayments",
        "Total Charges/Interest Paid"
    ]

    df = df[df["Country"].notna() & (df["Country"].str.strip() != "")]

    for c in df.columns:
        if c != "Country":
            df[c] = (
                df[c].astype(str)
                    .str.replace(",", "", regex=False)
                    .str.replace("…", "", regex=False)
                    .str.replace("..", "", regex=False)
                    .str.strip()
            )
            df[c] = pd.to_numeric(df[c], errors="coerce")

    return df


def read_imf_proj_tsv(path: Path) -> pd.DataFrame:
    """
    PROJECTIONS (extfor2): country-level schedule for one month.
    Parse starting at header line that begins with 'Member\t'.
    """
    text = path.read_text(encoding="utf-8", errors="replace")
    lines = text.splitlines()

    header_i = None
    for i, line in enumerate(lines):
        clean = line.lstrip("\ufeff").strip()
        if clean.startswith("Member\t"):
            header_i = i
            break

    if header_i is None:
        raise ValueError(f"Could not find projection header starting with 'Member\\t' in {path.name}")

    table_text = "\n".join(lines[header_i:])
    df = pd.read_csv(StringIO(table_text), sep="\t", dtype=str)

    # standardize country name
    df = df.rename(columns={"Member": "Country"})
    df = df[df["Country"].notna() & (df["Country"].str.strip() != "")]

    # numeric conversion for all non-country cols
    for c in df.columns:
        if c != "Country":
            df[c] = (
                df[c].astype(str)
                    .str.replace(",", "", regex=False)
                    .str.replace("…", "", regex=False)
                    .str.replace("..", "", regex=False)
                    .str.strip()
            )
            df[c] = pd.to_numeric(df[c], errors="coerce")

    return df

# ======================
# 1) 2025 monthly ACTUALS (country-level panel)
# ======================
all_2025 = []
for m in range(1, 13):
    f = folder / f"2025_{m:02d}.tsv"
    if not f.exists():
        print(f"⚠️ Missing: {f.name}")
        continue

    print(f"Processing ACTUAL {f.name}")
    df = read_imf_actual_tsv(f)
    df["Month"] = f"2025-{m:02d}"
    all_2025.append(df)

df_2025 = pd.concat(all_2025, ignore_index=True)
out1 = outdir / "imf_country_monthly_actual_2025.csv"
df_2025.to_csv(out1, index=False)
print(f"✅ Saved: {out1}")

# ======================
# 2) Jan 2026 ACTUAL (country-level)
# ======================
f_jan26 = folder / "2026_01.tsv"
print(f"Processing ACTUAL {f_jan26.name}")
df_jan26 = read_imf_actual_tsv(f_jan26)
df_jan26["Month"] = "2026-01"

out2 = outdir / "imf_country_actual_2026_01.csv"
df_jan26.to_csv(out2, index=False)
print(f"✅ Saved: {out2}")

# ======================
# 3) Feb–Dec 2026 PROJECTIONS aggregated to YEAR TOTAL (country-level)
# ======================
proj_parts = []
for m in range(2, 13):
    f = folder / f"2026_{m:02d}_proj.tsv"
    if not f.exists():
        print(f"⚠️ Missing: {f.name}")
        continue

    print(f"Processing PROJ {f.name}")
    df = read_imf_proj_tsv(f)
    df["Month"] = f"2026-{m:02d}"
    proj_parts.append(df)

df_proj = pd.concat(proj_parts, ignore_index=True)

# Sum across months for each country (Feb–Dec)
num_cols = [c for c in df_proj.columns if c not in ["Country", "Month"]]
df_proj_year = (
    df_proj.groupby("Country", as_index=False)[num_cols]
           .sum(numeric_only=True)
)
df_proj_year["Period"] = "2026-02_to_2026-12"

out3 = outdir / "imf_country_projected_2026_feb_dec_YEAR_TOTAL.csv"
df_proj_year.to_csv(out3, index=False)
print(f"✅ Saved: {out3}")

Processing ACTUAL 2025_01.tsv
Processing ACTUAL 2025_02.tsv
Processing ACTUAL 2025_03.tsv
Processing ACTUAL 2025_04.tsv
Processing ACTUAL 2025_05.tsv
Processing ACTUAL 2025_06.tsv
Processing ACTUAL 2025_07.tsv
Processing ACTUAL 2025_08.tsv
Processing ACTUAL 2025_09.tsv
Processing ACTUAL 2025_10.tsv
Processing ACTUAL 2025_11.tsv
Processing ACTUAL 2025_12.tsv
✅ Saved: /Users/paolajaimes/Downloads/imf_country_monthly_actual_2025.csv
Processing ACTUAL 2026_01.tsv
✅ Saved: /Users/paolajaimes/Downloads/imf_country_actual_2026_01.csv
Processing PROJ 2026_02_proj.tsv
Processing PROJ 2026_03_proj.tsv
Processing PROJ 2026_04_proj.tsv
Processing PROJ 2026_05_proj.tsv
Processing PROJ 2026_06_proj.tsv
Processing PROJ 2026_07_proj.tsv
Processing PROJ 2026_08_proj.tsv
Processing PROJ 2026_09_proj.tsv
Processing PROJ 2026_10_proj.tsv
Processing PROJ 2026_11_proj.tsv
Processing PROJ 2026_12_proj.tsv
✅ Saved: /Users/paolajaimes/Downloads/imf_country_projected_2026_feb_dec_YEAR_TOTAL.csv
